## 1) Optimizing the Dollar Sales by varying the EDLP / TPR prices using Solvers

In [2]:
import pandas,pyomo,numpy as np,pandas as pd
from pyomo.environ import *
import pyomo as pyo

In [6]:
sales = pd.DataFrame({"Week":["W1","W2","W3"],"Base":[10,10,10],"EDLP Price":[9.5,9.5,9.5],"TPR Discount":[0,10,20]})

edlp_coef = -1
tpr_coef = 4

sales["Unit Sales"]=sales['EDLP Price']*edlp_coef + sales['TPR Discount']*tpr_coef + sales["Base"]
sales["Price"]=sales['EDLP Price']*(1-(sales['TPR Discount']/100))
sales["Dollar Sales"] = sales["Unit Sales"] * sales["Price"]
sales["Trade Spend"] = sales["Base"] - sales["Price"]
sales["Total Trade Spend"] = sales["Trade Spend"] * sales["Unit Sales"]

sales_new = pd.DataFrame(sales)
sales_new

,Week,Base,EDLP Price,TPR Discount,Unit Sales,Price,Dollar Sales,Trade Spend,Total Trade Spend
0,W1,10,9.5,0,0.5,9.50,4.750,0.50,0.250
1,W2,10,9.5,10,40.5,8.55,346.275,1.45,58.725
2,W3,10,9.5,20,80.5,7.60,611.800,2.40,193.200


In [8]:
# Creating the Model using Pyomo and Solve the Objective

# Objective is been set as normal Model equation

model = ConcreteModel()

model.Locations = range(3)

model.x = Var(model.Locations, bounds=(9.5, 10) )
model.y = Var(model.Locations, bounds=(0, 50) )

model.obj = Objective(expr= sum(model.x[n] *(1-model.y[n]/100)* (10+model.x[n] *-1+model.y[n]*4) for n in model.Locations),
                      sense= maximize )

model.c = Constraint( expr = sum((10-model.x[n] *(1-model.y[n]/100)) * (10+model.x[n] *-1+model.y[n]*4) 
                                 for n in model.Locations)<=154.654)

opt = pyo.opt.SolverFactory('ipopt')

#PICOprint(model.display())
opt.solve(model,tee=False)
print(model.display())

Model unknown

  Variables:
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :   9.5 :  10.0 :    10 : False : False :  Reals
          1 :   9.5 :  10.0 :    10 : False : False :  Reals
          2 :   9.5 :  10.0 :    10 : False : False :  Reals
    y : Size=3, Index=y_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :     0 : 11.35245986416539 :    50 : False : False :  Reals
          1 :     0 : 11.35245986416539 :    50 : False : False :  Reals
          2 :     0 : 11.35245986416539 :    50 : False : False :  Reals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 1207.6411697388635

  Constraints:
    c : Size=1
        Key  : Lower : Body               : Upper
        None :  None : 154.65401396098326 : 154.654
None


In [9]:
## Creating the model with new co-efficients and log-exponential form of Model equation and the constraint accordingly

model = ConcreteModel()

model.Locations = range(3)

model.x = Var(model.Locations, bounds=(9.5, 10) )
model.y = Var(model.Locations, bounds=(0, 50) )

model.obj = Objective(expr= sum( (model.x[n] * (1-model.y[n]/100)) * (exp( 3+ (-1)* log(model.x[n]) + model.y[n]*0.4))
                                for n in model.Locations),
                      sense= maximize )

model.c = Constraint( expr = sum( (10 - model.x[n] * (1 - model.y[n]/100) ) * exp( 3+ log(model.x[n]) *-1 
                                                                                  + model.y[n]*0.4 ) 
                                 for n in model.Locations ) <=1438.275) 

opt = pyo.opt.SolverFactory('bonmin')

#PICOprint(model.display())
opt.solve(model,tee=False)
print(model.display())

Model unknown

  Variables:
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :   9.5 :  10.0 :    10 : False : False :  Reals
          1 :   9.5 :  10.0 :    10 : False : False :  Reals
          2 :   9.5 :  10.0 :    10 : False : False :  Reals
    y : Size=3, Index=y_index
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     0 : 13.026860759632454 :    50 : False : False :  Reals
          1 :     0 :  13.02686075963245 :    50 : False : False :  Reals
          2 :     0 : 13.026860759632454 :    50 : False : False :  Reals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 9602.56695781853

  Constraints:
    c : Size=1
        Key  : Lower : Body               : Upper
        None :  None : 1438.2751247926649 : 1438.275
None


In [10]:
# Solved the same model using ipopt

from pyomo.environ import *
import pyomo as pyo
model = ConcreteModel()

model.Locations = range(3)

model.x = Var(model.Locations, bounds=(9.5, 10) )
model.y = Var(model.Locations, bounds=(0, 50) )

model.obj = Objective(expr= sum( (model.x[n] * (1-model.y[n]/100)) * (exp( 3+ (-1)* log(model.x[n]) + model.y[n]*0.4))
                                for n in model.Locations),
                      sense= maximize )

model.c = Constraint( expr = sum( (10 - model.x[n] * (1 - model.y[n]/100) ) * exp( 3+ log(model.x[n]) *-1 
                                                                                  + model.y[n]*0.4 ) 
                                 for n in model.Locations ) <=1438.275) 

opt = pyo.opt.SolverFactory('ipopt')

#PICOprint(model.display())
opt.solve(model,tee=False)
print(model.display())

Model unknown

  Variables:
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :   9.5 :  10.0 :    10 : False : False :  Reals
          1 :   9.5 :  10.0 :    10 : False : False :  Reals
          2 :   9.5 :  10.0 :    10 : False : False :  Reals
    y : Size=3, Index=y_index
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :     0 : 13.026860759627581 :    50 : False : False :  Reals
          1 :     0 : 13.026860759627581 :    50 : False : False :  Reals
          2 :     0 : 13.026860759627581 :    50 : False : False :  Reals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 9602.566957800364

  Constraints:
    c : Size=1
        Key  : Lower : Body               : Upper
        None :  None : 1438.2751247893239 : 1438.275
None


### 2) Introducing the FLAG Variable for keeping one of the variables fixed

In [11]:
from pyomo.environ import *
import pyomo.environ as pyen
import pyomo as pyo
model = ConcreteModel()

model.Locations = range(3)

model.x = Var(model.Locations, bounds=(9.5, 10) )
model.y = Var(model.Locations, bounds=(0, 50) )
model.z = Var(model.Locations, within=Binary)

BP  = 10
EDLP = model.x
TPR = model.y
FLAG = model.z
coef_tpr = 0.004
coef_edlp = -3
itcpt =  15

model.obj = Objective(
    expr= sum(  ((BP*(1-TPR[n]/100)*(1-FLAG[n])) + (EDLP[n]*FLAG[n]) )
                                * ( exp( (log(EDLP[n])*coef_edlp)*FLAG[n] + (log(BP)*coef_edlp + TPR[n]*coef_tpr) +itcpt) )
                                for n in model.Locations),
                      sense= maximize )

model.overall = Constraint(expr = sum( ( exp( (log(EDLP[n])*coef_edlp)*FLAG[n] + (log(BP)*coef_edlp + TPR[n]*coef_tpr) +itcpt) ) *
                                      ((BP-EDLP[n])*FLAG[n] +(BP - BP*(1-TPR[n]/100))*(1-FLAG[n]) )
                                      for n in model.Locations ) <=10515) 

model.edlpspend = Constraint(expr = sum( (( exp( (log(EDLP[n])*coef_edlp)*FLAG[n] + (log(BP)*coef_edlp + TPR[n]*coef_tpr) +itcpt) ) *
                                      ((BP-EDLP[n])*FLAG[n] +(BP - BP*(1-TPR[n]/100))*(1-FLAG[n]) ))*FLAG[n]
                                      for n in model.Locations ) <=1906.410481) 

model.tprsend = Constraint(expr = sum( (( exp( (log(EDLP[n])*coef_edlp)*FLAG[n] + (log(BP)*coef_edlp + TPR[n]*coef_tpr) +itcpt) ) *
                                      ((BP-EDLP[n])*FLAG[n] +(BP - BP*(1-TPR[n]/100))*(1-FLAG[n]) ))* (1-FLAG[n])
                                      for n in model.Locations ) <=8609.171678 ) 

opt = pyo.opt.SolverFactory('ipopt')

#PICOprint(model.display())
opt.solve(model,tee=False)
print(model.display())

Model unknown

  Variables:
    x : Size=3, Index=x_index
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
          0 :   9.5 : 9.929354922356898 :    10 : False : False :  Reals
          1 :   9.5 : 9.929354922356898 :    10 : False : False :  Reals
          2 :   9.5 : 9.929354922356957 :    10 : False : False :  Reals
    y : Size=3, Index=y_index
        Key : Lower : Value                  : Upper : Fixed : Stale : Domain
          0 :     0 : 1.8416423456343914e-08 :    50 : False : False :  Reals
          1 :     0 : 1.8416423456343914e-08 :    50 : False : False :  Reals
          2 :     0 : 1.8416423456343914e-08 :    50 : False : False :  Reals
    z : Size=3, Index=z_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    obj : Size=1, I

In [13]:
# Using Bonmin

base_price=[10,10,10]
target_trade_spend = 10515.58
intercept = 15
coef_1 =-3
coef_2 =0.004
target_edlp_spend = 1906.41
target_tpr_spend = 8609.172

model = ConcreteModel(name='Spend_Optim')
idx = range(3)
model.idx = idx
model.edlp = Var(idx, initialize=10, bounds=(9.5, 10))
model.tpr = Var(idx, initialize=10, bounds=(5, 50))

# model.S = Set(initialize={1,2,3})
# model.x = Var(model.S)
model.flag = Var(idx,initialize=0,domain=Binary)



unit_sales = list(exp((log(model.edlp[i])*coef_1)*model.flag[i]
                 +(log(base_price[i])*coef_1+model.tpr[i]*coef_2)*(1-model.flag[i])
                 +intercept) for i in model.idx)

price = list((base_price[i]*(1-model.tpr[i]/100)*(1-model.flag[i])) 
        +(model.edlp[i]*model.flag[i]) for i in model.idx)

dollar_sales = [a*b for a,b in zip(unit_sales,price)]

trade_spent = [a-b for a,b in zip(base_price, price)]

total_trade_spent = [a*b for a,b in zip(trade_spent, unit_sales)]

edlp_trade_spent = [a*b for a,b in zip(total_trade_spent, model.flag)]

tpr_trade_spent = [a*(1-b) for a,b in zip(total_trade_spent, model.flag)]

flag_util = [a*(1-b) for a,b in zip(model.flag, model.flag)]

model.obj = Objective(expr=sum(dollar_sales) ,sense= maximize)


model.c1 = Constraint(expr = sum(total_trade_spent) == target_trade_spend )
model.c2 = Constraint(expr = sum(edlp_trade_spent) == target_edlp_spend )
model.c3 = Constraint(expr = sum(tpr_trade_spent) == target_tpr_spend )
model.c4 = Constraint(expr = (model.flag[0]*(1-model.flag[0]) ==0) )
model.c5 = Constraint(expr = (model.flag[1]*(1-model.flag[1]) ==0) )
model.c6 = Constraint(expr = (model.flag[2]*(1-model.flag[2]) ==0) )
opt = SolverFactory('bonmin')
opt.solve(model)
model.display()

    model=Spend_Optim;
        message from solver=bonmin\x3a Infeasible problem
Model Spend_Optim

  Variables:
    edlp : Size=3, Index=edlp_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :   9.5 :    10 :    10 : False :  True :  Reals
          1 :   9.5 :    10 :    10 : False :  True :  Reals
          2 :   9.5 :    10 :    10 : False :  True :  Reals
    tpr : Size=3, Index=tpr_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     5 :    10 :    50 : False :  True :  Reals
          1 :     5 :    10 :    50 : False :  True :  Reals
          2 :     5 :    10 :    50 : False :  True :  Reals
    flag : Size=3, Index=flag_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :     0 :     1 : False :  True : Binary
          1 :     0 :     0 :     1 : False :  True : Binary
          2 :     0 :     0 :     1 : False :  True : Binary

  Objectives:
    obj : Size=1, Index=None, Acti

#### 3) Created the Problem and had set the constraints

In [14]:
# Func wise

from __future__ import division
from pyomo.environ import *
import numpy as np
import pandas as pd
from itertools import combinations as cmb
import time

No_Of_Weeks = 3
Base_Price = [10] * No_Of_Weeks
Target_Trade_Spend = 10515.58
Target_EDLP_Spend = 1906.41
Target_TPR_Spend = 8609.172
Intercept = 15
EDLP_Coef = -3
TPR_Coef = 0.004
Init_List = list(set(cmb([0,0,0,1,1,1],3)))
Model.Location = range(3)
LB = [9.5,9.5,9.5,5,5,5,0,0,0]
UB = [10,10,10,50,50,50,1,1,1]

def Initialize_Fnc(Model,i):
    return Init_List[j][i]

# i --> range(3) as iterations
# Variables[i]                  --> EDLP (index 0,1,2) 
# Variables[i + No_Of_Weeks]    --> TPR  (index 3,4,5)
# Variables[i + 2*No_Of_Weeks]  --> FLAG (index 6,7,8)

def Unit_Sales_Fnc(Variables):
    
    Unit_Sales = [exp( (log(Variables[i]) * EDLP_Coef) * Variables[i + 2*No_Of_Weeks]
                 +(log(Base_Price[i]) * EDLP_Coef + Variables[i + No_Of_Weeks] * TPR_Coef) * (1- Variables[i + 2*No_Of_Weeks])
                 +Intercept ) for i in  Model.Location]
    
    return Unit_Sales
    
def Sales_Price_Fnc(Variables):
    
    Sales_Price = [ (Base_Price[i]* (1-Variables[i + No_Of_Weeks]/100) * (1-Variables[i + 2*No_Of_Weeks]))
        +(Variables[i] * Variables[i + 2*No_Of_Weeks]) for i in Model.Location]

    return Sales_Price

def Dollar_Sales_Fnc(Variables):
  
    Unit_Sales, Sales_Price = Unit_Sales_Fnc(Variables), Sales_Price_Fnc(Variables)
   
    Dollar_Sales = [Unit_Sales[i] * Sales_Price[i] for i in Model.Location]
   
    return Dollar_Sales

def Actual_Total_Trade_Spent_Fnc(Variables):
   
    Unit_Sales, Sales_Price = Unit_Sales_Fnc(Variables), Sales_Price_Fnc(Variables)
    
    Trade_Spent = [Base_Price[i]-Sales_Price[i] for i in Model.Location]
    
    Actual_Total_Trade_Spent = [Trade_Spent[i] * Unit_Sales[i] for i in Model.Location]
   
    return Actual_Total_Trade_Spent

def Actual_EDLP_Trade_Spent_Fnc(Variables):
    
    Unit_Sales, Sales_Price = Unit_Sales_Fnc(Variables), Sales_Price_Fnc(Variables)
    
    Trade_Spent = [Base_Price[i]-Sales_Price[i] for i in Model.Location]
   
    Actual_Total_Trade_Spent = [Trade_Spent[i] * Unit_Sales[i] for i in Model.Location]
   
    Actual_EDLP_Trade_Spent = [Actual_Total_Trade_Spent[i] * Variables[i + 2*No_Of_Weeks] for i in Model.Location]
   
    return Actual_EDLP_Trade_Spent

def Actual_TPR_Trade_Spent_Fnc(Variables):
    
    Unit_Sales, Sales_Price = Unit_Sales_Fnc(Variables), Sales_Price_Fnc(Variables)
    
    Trade_Spent = [Base_Price[i]-Sales_Price[i] for i in Model.Location]
   
    Actual_Total_Trade_Spent = [Trade_Spent[i] * Unit_Sales[i] for i in Model.Location]
   
    Actual_TPR_Trade_Spent = [Actual_Total_Trade_Spent[i] * (1 - Variables[i + 2*No_Of_Weeks]) for i in Model.Location]
   
    return Actual_TPR_Trade_Spent

Model = ConcreteModel()

Model.Slot = Set(initialize=list(range(9)))

# Binary values for FLAG and for the rest NonNegativeReals

def variable_Domain_Set_Fnc(Model,i):

    if i < 2 * No_Of_Weeks:
    
        return NonNegativeReals
    
    else:
        
        return Binary

def Boundary_Set_Fnc(Model,i):
    return (LB[i],UB[i])


def Variable_Initialization_Fnc(Model,i):
    
    if i < No_Of_Weeks: # EDLP --> initialization as Upper Boundary
    
        return 10   
    
    if i< 2 * No_Of_Weeks: # TPR --> initialization as lower boundary 
            
        return 5
    
    elif i==7:  # FLAG --> initialization as  Binary
    
        return 1

    elif i==8:  # FLAG --> initialization as  Binary
    
        return 1
    
    else:
        
        return 0


Model.Variables = Var(Model.Slot, within=variable_Domain_Set_Fnc, bounds=Boundary_Set_Fnc, 
                      initialize= Variable_Initialization_Fnc)


Model.Obj = Objective(expr=sum(Dollar_Sales_Fnc(Model.Variables)) ,sense= maximize)

Model.OverAll = Constraint(expr = sum(Actual_Total_Trade_Spent_Fnc(Model.Variables)) == Target_Trade_Spend )


Model.EDLP = Constraint(expr = sum(Actual_EDLP_Trade_Spent_Fnc(Model.Variables)) == Target_EDLP_Spend )


# Model.TPR = Constraint(expr = sum(Actual_TPR_Trade_Spent_Fnc(Model.Variables)) == Target_TPR_Spend )


start_time=time.time()
opt_1 = SolverFactory('bonmin')
opt_1.solve(Model)
end_time=time.time()
Model.display()
print("\n\n The Elapsed Time is --- %s Seconds ---" % (end_time - start_time))

Model unknown

  Variables:
    Variables : Size=9, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :  9.800160744235571 :    10 : False : False : NonNegativeReals
          1 :   9.5 :  9.731292000180645 :    10 : False : False : NonNegativeReals
          2 :   9.5 :  9.731292000180645 :    10 : False : False : NonNegativeReals
          3 :     5 : 23.931543297945627 :    50 : False : False : NonNegativeReals
          4 :     5 :  35.58828025395619 :    50 : False : False : NonNegativeReals
          5 :     5 :  35.58828025395619 :    50 : False : False : NonNegativeReals
          6 :     0 :                0.0 :     1 : False : False :           Binary
          7 :     0 :                1.0 :     1 : False : False :           Binary
          8 :     0 :                1.0 :     1 : False : False :           Binary

  Objectives:
    Obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   Tru

### Two Products and 3 weeks

In [15]:
# Two Products, 3 weeks

from __future__ import division
from pyomo.environ import *
import numpy as np
import pandas as pd
import copy
from itertools import combinations as cmb
import time

Base_Price = [[10,10,10],[10,10,10]]
Intercepts = [23,20]

P1_EDLP_coef = [-6,1.2]
P2_EDLP_coef = [1,-5]

P1_TPR_coef = [0.01,-0.004]
P2_TPR_coef = [-0.003,0.02]

Target_Trade_Spend = [276860,223573]
Target_EDLP_Spend = [63366,35525]
Target_TPR_Spend = [213494,188048]

Model.Locations = range(3)

No_Of_Products = 2
No_Of_Weeks = 3

Total = No_Of_Products * No_Of_Weeks

Init_List = list(set(cmb([0,0,0,1,1,1],3)))
Model.Location = range(3)

LB = [9.5,9.5,9.5,9.5,9.5,9.5,5,5,5,5,5,5,0,0,0,0,0,0]
UB = [10,10,10,10,10,10,50,50,50,50,50,50,1,1,1,1,1,1]

def Initialize_Fnc(Model,i):
    return Init_List[i]
# i --> range(6) as iterations
# Variables[i]                  --> EDLP (index 0,1,..5) 
# Variables[i + No_Of_Weeks*No_of_Products]    --> TPR  (index 6,7..11)
# Variables[i + 2*No_of_Products]  --> FLAG (index 12,13,..17)

def Unit_Sales_Price_Fnc(Variables):
        
    Unit_Sales = []
    Sales_Price= []

    for j in range(No_Of_Products):

        Unit_Sales.append([exp(((log(Variables[i])*P1_EDLP_coef[j])* Variables[i+2*No_Of_Products*No_Of_Weeks])
                               +((log(Base_Price[j][i])*P1_EDLP_coef[j]+
                                  Variables[i+No_Of_Products*No_Of_Weeks]*P1_TPR_coef[j]) * 
                                 (1- Variables[i+2*No_Of_Products*No_Of_Weeks]))
                               +((log(Variables[i+No_Of_Weeks])*P2_EDLP_coef[j])* 
                                 Variables[i+2*No_Of_Products*No_Of_Weeks+No_Of_Weeks])
                               +((log(Base_Price[j][i])*P2_EDLP_coef[j]+ Variables[i+No_Of_Products*No_Of_Weeks+
                                                                                   No_Of_Weeks]*
             P2_TPR_coef[j])*(1- Variables[i+2*No_Of_Products*No_Of_Weeks+No_Of_Weeks]))+Intercepts[j]) for i in Model.Locations])

        Sales_Price.append([ (Base_Price[j][i]* (1-Variables[i + No_Of_Weeks*No_Of_Products+j*No_Of_Weeks]/100) *
                         (1-Variables[i + 2*No_Of_Weeks*No_Of_Products+j*No_Of_Weeks]))+(Variables[i+j*No_Of_Weeks] * 
            Variables[i + 2*No_Of_Weeks*No_Of_Products+j*No_Of_Weeks]) for i in Model.Locations])

    return Unit_Sales,Sales_Price
            
def Dollar_Sales_Fnc(Variables):
    
    Dollar_Sales=[]
    
    Unit_Sales, Price = Unit_Sales_Price_Fnc(Variables)
    
    for j in range(No_Of_Products):
    
        Dollar_Sales_Single = [Unit_Sales[j][i]*Price[j][i] for i in Model.Locations]
        
        Dollar_Sales.append(Dollar_Sales_Single)
   
    return Dollar_Sales


def Actual_Total_Trade_Spent_Fnc(Variables):
   
    Unit_Sales, Sales_Price = Unit_Sales_Price_Fnc(Variables)
    
    Actual_Total_Trade_Spent = []
    
    for j in range(No_Of_Products):
    
        Trade_Spent = [Base_Price[j][i]-Sales_Price[j][i] for i in Model.Locations]

        Actual_Total_Trade_Spent.append([Trade_Spent[i] * Unit_Sales[j][i] for i in Model.Locations])    

    return Actual_Total_Trade_Spent

def Actual_EDLP_Trade_Spent_Fnc(Variables):
    
    Unit_Sales, Price = Unit_Sales_Price_Fnc(Variables)
    
    Actual_Total_Trade_Spent = Actual_Total_Trade_Spent_Fnc(Variables)
    
    Actual_EDLP_Trade_Spent = []
    
    for j in range(No_Of_Products):
        
        Actual_EDLP_Trade_Spent.append([Actual_Total_Trade_Spent[j][i] * 
                                        Variables[i + 2*No_Of_Weeks*No_Of_Products+j*No_Of_Weeks] for i in Model.Locations])
   
    return Actual_EDLP_Trade_Spent

def Actual_TPR_Trade_Spent_Fnc(Variables):
    
    Unit_Sales, Sales_Price = Unit_Sales_Fnc(Variables), Sales_Price_Fnc(Variables)
    
    Actual_TPR_Trade_Spent = []

    Actual_Total_Trade_Spent = Actual_Total_Trade_Spent(Variables)
    
    for j in range(No_Of_Products):
        
        Actual_TPR_Trade_Spent.append([Actual_Total_Trade_Spent[j][i] * 
                                       (1 - Variables[i+2*No_Of_Weeks*No_Of_Weeks+j*No_Of_Weeks]) for i in Model.Locations])
   
    return Actual_TPR_Trade_Spent


Model = ConcreteModel(name='Spend_Optim')

Model.Slot = Set(initialize=list(range(18)))

# Binary values for FLAG and for the rest NonNegativeReals

def variable_Domain_Set_Fnc(Model,i):

    if i < 2 * No_Of_Weeks * No_Of_Products:
    
        return NonNegativeReals
    
    else:
        
        return Binary

def Boundary_Set_Fnc(Model,i):
    
    return (LB[i],UB[i])

init=(0,1,0,1,0,1)
def Variable_Initialization_Fnc(Model,i,m=init):
    
    m,n,o,p,q,r = init
    
    if i < No_Of_Weeks * No_Of_Products: # EDLP --> initialization as Upper Boundary
    
        return 10   
    
    if i < 2 * No_Of_Weeks * No_Of_Products: # TPR --> initialization as lower boundary 
            
        return 5
    
    elif i == 12:  # FLAG --> initialization as  Binary
    
        return m

    elif i == 13:  # FLAG --> initialization as  Binary
    
        return n

    elif i == 14:  # FLAG --> initialization as  Binary
    
        return o

    elif i == 15:  # FLAG --> initialization as  Binary
    
        return p

    elif i == 16:  # FLAG --> initialization as  Binary
    
        return q

    elif i == 17:  # FLAG --> initialization as  Binary
    
        return r

init=(1,1,1,1,1,1)
Model.Variables = Var(Model.Slot, within=variable_Domain_Set_Fnc, bounds=Boundary_Set_Fnc, 
                      initialize= Variable_Initialization_Fnc)


Dollar_Sales = Dollar_Sales_Fnc(Model.Variables)

Total_Trade_Spent = Actual_Total_Trade_Spent_Fnc(Model.Variables)

EDLP_Trade_Spent = Actual_EDLP_Trade_Spent_Fnc(Model.Variables)

Model.Obj = Objective(expr=sum(sum(Dollar_Sales[i]) for i in range(len(Dollar_Sales))) ,sense= maximize)

for i in range(No_Of_Products): # Constraints for the Products
    
    exec(f"Model.OverAll_P{i+1} = Constraint(expr = sum(Total_Trade_Spent[{i}]) == Target_Trade_Spend[{i}])")
    exec(f"Model.EDLP_P{i+1} = Constraint(expr = sum(EDLP_Trade_Spent[{i}]) == Target_EDLP_Spend[{i}])" )
    
start_time = time.time()
opt_1 = SolverFactory('bonmin')
results_1=opt_1.solve(Model)
end_time = time.time()
Model.display()
print("\n\n The Elapsed Time is --- %s Seconds ---" % (end_time - start_time))

Model Spend_Optim

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :  9.501569774312637 :    10 : False : False : NonNegativeReals
          1 :   9.5 :  9.932987433582595 :    10 : False : False : NonNegativeReals
          2 :   9.5 :  9.932987433582595 :    10 : False : False : NonNegativeReals
          3 :   9.5 :   9.59955298210853 :    10 : False : False : NonNegativeReals
          4 :   9.5 :  9.932987433582595 :    10 : False : False : NonNegativeReals
          5 :   9.5 :  9.932987433582595 :    10 : False : False : NonNegativeReals
          6 :     5 :  49.47313875361004 :    50 : False : False : NonNegativeReals
          7 :     5 : 10.203736454217939 :    50 : False : False : NonNegativeReals
          8 :     5 : 10.203736454217939 :    50 : False : False : NonNegativeReals
          9 :     5 :  49.47313875361004 :    50 : False : False : NonNegativeReals
         10 :     

In [16]:
# Comparing the time taken and the objectives found for the multiple solvers

In [17]:
combo = list(set(cmb([0,0,0,1,1,1],3)))
combo.sort()

In [18]:
for name in ('bonmin','ipopt','bnb'):
    for comb in combo:
        for com in combo:
            init=comb+com
                
            
            Model.Variables = Var(Model.Slot, within=variable_Domain_Set_Fnc, bounds=Boundary_Set_Fnc, 
                                  initialize= Variable_Initialization_Fnc)


            Dollar_Sales = Dollar_Sales_Fnc(Model.Variables)

            Total_Trade_Spent = Actual_Total_Trade_Spent_Fnc(Model.Variables)

            EDLP_Trade_Spent = Actual_EDLP_Trade_Spent_Fnc(Model.Variables)

            Model.Obj = Objective(expr=sum(sum(Dollar_Sales[i]) for i in range(len(Dollar_Sales))) ,sense= maximize)

            for i in range(No_Of_Products): # Constraints for the Products

                exec(f"Model.OverAll_P{i+1} = Constraint(expr = sum(Total_Trade_Spent[{i}]) == Target_Trade_Spend[{i}])")
                exec(f"Model.EDLP_P{i+1} = Constraint(expr = sum(EDLP_Trade_Spent[{i}]) == Target_EDLP_Spend[{i}])" )

            start_time = time.time()
            opt_1 = SolverFactory(name)
            result = opt_1.solve(Model)
            end_time = time.time()
            print(f"\n SOLVER Name : {name} \n")
            print(f"\n P1 Initialization Value: {comb} , P2 Initialization value: {com} \n ")
            Model.display()
            print("\n\nTime taken to execute the code is --- %s Seconds ---" % (end_time - start_time),"\n\n")
            print(f"\n{name} Solver Execution Time: {result['Solver'][0]['Time']}\n\n")


    'pyomo.core.base.var.IndexedVar'>) on block Spend_Optim with a new
    Component (type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.objective.SimpleObjective'>) on block Spend_Optim with a
    new Component (type=<class 'pyomo.core.base.objective.SimpleObjective'>).
    This is usually indicative of a modelling error. To avoid this warning,
    use block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on block Spend_Optim with
    a new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.SimpleConstraint'>) on block Spend_Optim with
    a new Component (type=<class
    'pyomo.core.base.constraint.Simp


 SOLVER Name : bonmin 


 P1 Initialization Value: (0, 0, 0) , P2 Initialization value: (0, 1, 1) 
 
Model Spend_Optim

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :   9.50563219866149 :    10 : False : False : NonNegativeReals
          1 :   9.5 :       9.7500000025 :    10 : False : False : NonNegativeReals
          2 :   9.5 :       9.7500000025 :    10 : False : False : NonNegativeReals
          3 :   9.5 :       9.7500000025 :    10 : False : False : NonNegativeReals
          4 :   9.5 :  9.607149465629808 :    10 : False : False : NonNegativeReals
          5 :   9.5 :       9.7500000025 :    10 : False : False : NonNegativeReals
          6 :     5 : 27.500000225000004 :    50 : False : False : NonNegativeReals
          7 :     5 :  9.561578253214936 :    50 : False : False : NonNegativeReals
          8 :     5 :  10.92921893893211 :    50 : False : False : NonNegativeReals



 SOLVER Name : bonmin 


 P1 Initialization Value: (0, 0, 1) , P2 Initialization value: (0, 0, 0) 
 
Model Spend_Optim

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :       9.7500000025 :    10 : False : False : NonNegativeReals
          1 :   9.5 :       9.7500000025 :    10 : False : False : NonNegativeReals
          2 :   9.5 :   9.50563219866149 :    10 : False : False : NonNegativeReals
          3 :   9.5 :  9.607149465629806 :    10 : False : False : NonNegativeReals
          4 :   9.5 :       9.7500000025 :    10 : False : False : NonNegativeReals
          5 :   9.5 :       9.7500000025 :    10 : False : False : NonNegativeReals
          6 :     5 :   9.56157825321527 :    50 : False : False : NonNegativeReals
          7 :     5 : 10.929218938931774 :    50 : False : False : NonNegativeReals
          8 :     5 :       27.500000225 :    50 : False : False : NonNegativeReals



 SOLVER Name : bonmin 


 P1 Initialization Value: (0, 0, 1) , P2 Initialization value: (0, 1, 1) 
 
Model Spend_Optim

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :   9.73818244545862 :    10 : False : False : NonNegativeReals
          1 :   9.5 :   9.63084527906831 :    10 : False : False : NonNegativeReals
          2 :   9.5 :  9.693612072786639 :    10 : False : False : NonNegativeReals
          3 :   9.5 :   9.63084527906831 :    10 : False : False : NonNegativeReals
          4 :   9.5 :   9.84264175017696 :    10 : False : False : NonNegativeReals
          5 :   9.5 :     9.725592787191 :    10 : False : False : NonNegativeReals
          6 :     5 :  44.78870281348583 :    50 : False : False : NonNegativeReals
          7 :     5 :  18.49945018231484 :    50 : False : False : NonNegativeReals
          8 :     5 : 10.519336881692865 :    50 : False : False : NonNegativeReals



 SOLVER Name : bonmin 


 P1 Initialization Value: (0, 1, 1) , P2 Initialization value: (0, 0, 0) 
 
Model Spend_Optim

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :   9.74780556535855 :    10 : False : False : NonNegativeReals
          1 :   9.5 :  9.714144893286122 :    10 : False : False : NonNegativeReals
          2 :   9.5 :  9.718419026628053 :    10 : False : False : NonNegativeReals
          3 :   9.5 :       9.7500000025 :    10 : False : False : NonNegativeReals
          4 :   9.5 :  9.750000002499998 :    10 : False : False : NonNegativeReals
          5 :   9.5 :  9.608443148172155 :    10 : False : False : NonNegativeReals
          6 :     5 : 18.716150815900466 :    50 : False : False : NonNegativeReals
          7 :     5 :  48.53276050010735 :    50 : False : False : NonNegativeReals
          8 :     5 :  48.53276050010735 :    50 : False : False : NonNegativeReals


...


 SOLVER Name : bnb 


 P1 Initialization Value: (0, 0, 1) , P2 Initialization value: (0, 1, 1) 
 
Model Spend_Optim

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :   9.73818244545801 :    10 : False : False : NonNegativeReals
          1 :   9.5 :               10.0 :    10 : False : False : NonNegativeReals
          2 :   9.5 :  9.693612072786724 :    10 : False : False : NonNegativeReals
          3 :   9.5 :  9.811568467801594 :    10 : False : False : NonNegativeReals
          4 :   9.5 :   9.84264175017915 :    10 : False : False : NonNegativeReals
          5 :   9.5 :  9.725592787189115 :    10 : False : False : NonNegativeReals
          6 :     5 :                5.0 :    50 : False : False : NonNegativeReals
          7 :     5 :  18.49945018231137 :    50 : False : False : NonNegativeReals
          8 :     5 : 15.712514017599815 :    50 : False : False : NonNegativeReals
   


 SOLVER Name : bnb 


 P1 Initialization Value: (0, 1, 1) , P2 Initialization value: (0, 0, 0) 
 
Model Spend_Optim

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :               10.0 :    10 : False : False : NonNegativeReals
          1 :   9.5 :  9.718419026627998 :    10 : False : False : NonNegativeReals
          2 :   9.5 :  9.714144893286171 :    10 : False : False : NonNegativeReals
          3 :   9.5 :                9.5 :    10 : False : False : NonNegativeReals
          4 :   9.5 :  9.608443148172151 :    10 : False : False : NonNegativeReals
          5 :   9.5 :               10.0 :    10 : False : False : NonNegativeReals
          6 :     5 : 18.716150815900146 :    50 : False : False : NonNegativeReals
          7 :     5 :                5.0 :    50 : False : False : NonNegativeReals
          8 :     5 :                5.0 :    50 : False : False : NonNegativeReals
   


 SOLVER Name : bnb 


 P1 Initialization Value: (0, 1, 1) , P2 Initialization value: (0, 1, 1) 
 
Model Spend_Optim

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :               10.0 :    10 : False : False : NonNegativeReals
          1 :   9.5 :  9.714144893286122 :    10 : False : False : NonNegativeReals
          2 :   9.5 :   9.71841902662806 :    10 : False : False : NonNegativeReals
          3 :   9.5 :                9.5 :    10 : False : False : NonNegativeReals
          4 :   9.5 :               10.0 :    10 : False : False : NonNegativeReals
          5 :   9.5 :  9.608443148172155 :    10 : False : False : NonNegativeReals
          6 :     5 : 18.716150815904168 :    50 : False : False : NonNegativeReals
          7 :     5 :                5.0 :    50 : False : False : NonNegativeReals
          8 :     5 :                5.0 :    50 : False : False : NonNegativeReals
   


 SOLVER Name : bnb 


 P1 Initialization Value: (1, 1, 1) , P2 Initialization value: (0, 0, 0) 
 
Model Spend_Optim

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :  9.615958216913123 :    10 : False : False : NonNegativeReals
          1 :   9.5 :  9.718419026628064 :    10 : False : False : NonNegativeReals
          2 :   9.5 :   9.71414489328612 :    10 : False : False : NonNegativeReals
          3 :   9.5 :                9.5 :    10 : False : False : NonNegativeReals
          4 :   9.5 :  9.608443148172155 :    10 : False : False : NonNegativeReals
          5 :   9.5 :               10.0 :    10 : False : False : NonNegativeReals
          6 :     5 : 18.716150815904395 :    50 : False : False : NonNegativeReals
          7 :     5 :                5.0 :    50 : False : False : NonNegativeReals
          8 :     5 :                5.0 :    50 : False : False : NonNegativeReals
   


 SOLVER Name : bnb 


 P1 Initialization Value: (1, 1, 1) , P2 Initialization value: (0, 1, 1) 
 
Model Spend_Optim

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :  9.718419026627796 :    10 : False : False : NonNegativeReals
          1 :   9.5 :  9.714144893286221 :    10 : False : False : NonNegativeReals
          2 :   9.5 :                9.5 :    10 : False : False : NonNegativeReals
          3 :   9.5 :  9.608443148172146 :    10 : False : False : NonNegativeReals
          4 :   9.5 :                9.5 :    10 : False : False : NonNegativeReals
          5 :   9.5 :               10.0 :    10 : False : False : NonNegativeReals
          6 :     5 :                5.0 :    50 : False : False : NonNegativeReals
          7 :     5 :                5.0 :    50 : False : False : NonNegativeReals
          8 :     5 : 18.716150815900093 :    50 : False : False : NonNegativeReals
   

### Added Buffer Constraints

In [19]:
# Two Products, 3 weeks

from __future__ import division
from pyomo.environ import *
import numpy as np
import pandas as pd
import copy
from itertools import combinations as cmb
import time

Base_Price = [[10,10,10],[10,10,10]]
Intercepts = [23,20]

P1_EDLP_coef = [-6,1.2]
P2_EDLP_coef = [1,-5]

P1_TPR_coef = [0.01,-0.004]
P2_TPR_coef = [-0.003,0.02]

Target_Trade_Spend = [276860,223573]
Target_EDLP_Spend = [63366,35525]
Target_TPR_Spend = [213494,188048]

Model.Locations = range(3)

No_Of_Products = 2
No_Of_Weeks = 3

Total = No_Of_Products * No_Of_Weeks

Init_List = list(set(cmb([0,0,0,1,1,1],3)))
Model.Location = range(3)

LB = [9.5,9.5,9.5,9.5,9.5,9.5,5,5,5,5,5,5,0,0,0,0,0,0]
UB = [10,10,10,10,10,10,50,50,50,50,50,50,1,1,1,1,1,1]

def Initialize_Fnc(Model,i):
    return Init_List[i]

# i --> range(6) as iterations
# Variables[i]                  --> EDLP (index 0,1,..5) 
# Variables[i + No_Of_Weeks*No_of_Products]    --> TPR  (index 6,7..11)
# Variables[i + 2*No_of_Products]  --> FLAG (index 12,13,..17)

def Unit_Sales_Price_Fnc(Variables):
        
    Unit_Sales = []
    Sales_Price= []

    for j in range(No_Of_Products):

        Unit_Sales.append([exp(((log(Variables[i])*P1_EDLP_coef[j])* Variables[i+2*No_Of_Products*No_Of_Weeks])
                               +((log(Base_Price[j][i])*P1_EDLP_coef[j]+
                                  Variables[i+No_Of_Products*No_Of_Weeks]*P1_TPR_coef[j]) * 
                                 (1- Variables[i+2*No_Of_Products*No_Of_Weeks]))
                               +((log(Variables[i+No_Of_Weeks])*P2_EDLP_coef[j])* 
                                 Variables[i+2*No_Of_Products*No_Of_Weeks+No_Of_Weeks])
                               +((log(Base_Price[j][i])*P2_EDLP_coef[j]+ Variables[i+No_Of_Products*No_Of_Weeks+
                                                                                   No_Of_Weeks]*
             P2_TPR_coef[j])*(1- Variables[i+2*No_Of_Products*No_Of_Weeks+No_Of_Weeks]))+Intercepts[j]) for i in Model.Locations])

        Sales_Price.append([ (Base_Price[j][i]* (1-Variables[i + No_Of_Weeks*No_Of_Products+j*No_Of_Weeks]/100) *
                         (1-Variables[i + 2*No_Of_Weeks*No_Of_Products+j*No_Of_Weeks]))+(Variables[i+j*No_Of_Weeks] * 
            Variables[i + 2*No_Of_Weeks*No_Of_Products+j*No_Of_Weeks]) for i in Model.Locations])

    return Unit_Sales,Sales_Price
            
def Dollar_Sales_Fnc(Variables):
    
    Dollar_Sales=[]
    
    Unit_Sales, Price = Unit_Sales_Price_Fnc(Variables)
    
    for j in range(No_Of_Products):
    
        Dollar_Sales_Single = [Unit_Sales[j][i]*Price[j][i] for i in Model.Locations]
        
        Dollar_Sales.append(Dollar_Sales_Single)
   
    return Dollar_Sales


def Actual_Total_Trade_Spent_Fnc(Variables):
   
    Unit_Sales, Sales_Price = Unit_Sales_Price_Fnc(Variables)
    
    Actual_Total_Trade_Spent = []
    
    for j in range(No_Of_Products):
    
        Trade_Spent = [Base_Price[j][i]-Sales_Price[j][i] for i in Model.Locations]

        Actual_Total_Trade_Spent.append([Trade_Spent[i] * Unit_Sales[j][i] for i in Model.Locations])    

    return Actual_Total_Trade_Spent

def Actual_EDLP_Trade_Spent_Fnc(Variables):
    
    Actual_Total_Trade_Spent = Actual_Total_Trade_Spent_Fnc(Variables)
    
    Actual_EDLP_Trade_Spent = []
    
    for j in range(No_Of_Products):
        
        Actual_EDLP_Trade_Spent.append([Actual_Total_Trade_Spent[j][i] * 
                                        Variables[i + 2*No_Of_Weeks*No_Of_Products+j*No_Of_Weeks] for i in Model.Locations])
   
    return Actual_EDLP_Trade_Spent

def Actual_TPR_Trade_Spent_Fnc(Variables):
    
    Actual_TPR_Trade_Spent = []

    Actual_Total_Trade_Spent = Actual_Total_Trade_Spent_Fnc(Variables)
    
    for j in range(No_Of_Products):
        
        Actual_TPR_Trade_Spent.append([Actual_Total_Trade_Spent[j][i] * 
                                        (1-Variables[i + 2*No_Of_Weeks*No_Of_Products+j*No_Of_Weeks]) for i in Model.Locations])
   
   
    return Actual_TPR_Trade_Spent


Model = ConcreteModel()

Model.Slot = Set(initialize=list(range(18)))

# Binary values for FLAG and for the rest NonNegativeReals

def variable_Domain_Set_Fnc(Model,i):

    if i < 2 * No_Of_Weeks * No_Of_Products:
    
        return NonNegativeReals
    
    else:
        
        return Binary

def Boundary_Set_Fnc(Model,i):
    
    return (LB[i],UB[i])

init=(0,1,0,1,0,1)
def Variable_Initialization_Fnc(Model,i,m=init):
    
    m,n,o,p,q,r = init
    
    if i < No_Of_Weeks * No_Of_Products: # EDLP --> initialization as Upper Boundary
    
        return 10   
    
    if i < 2 * No_Of_Weeks * No_Of_Products: # TPR --> initialization as lower boundary 
            
        return 5
    
    elif i == 12:  # FLAG --> initialization as  Binary
    
        return m

    elif i == 13:  # FLAG --> initialization as  Binary
    
        return n

    elif i == 14:  # FLAG --> initialization as  Binary
    
        return o

    elif i == 15:  # FLAG --> initialization as  Binary
    
        return p

    elif i == 16:  # FLAG --> initialization as  Binary
    
        return q

    elif i == 17:  # FLAG --> initialization as  Binary
    
        return r

init=(1,1,1,1,1,1)


combo = list(set(cmb([0,0,0,1,1,1],3)))
combo.sort()

for name in ('bonmin','ipopt','bnb'):
    for comb in combo:
        for com in combo:
            init=comb+com
                
            
            Model.Variables = Var(Model.Slot, within=variable_Domain_Set_Fnc, bounds=Boundary_Set_Fnc, 
                                  initialize= Variable_Initialization_Fnc)


            Dollar_Sales = Dollar_Sales_Fnc(Model.Variables)

            Total_Trade_Spent = Actual_Total_Trade_Spent_Fnc(Model.Variables)

            EDLP_Trade_Spent = Actual_EDLP_Trade_Spent_Fnc(Model.Variables)
            
            TPR_Trade_Spent = Actual_TPR_Trade_Spent_Fnc(Model.Variables)

            Model.Obj = Objective(expr=sum(sum(Dollar_Sales[i]) for i in range(len(Dollar_Sales))) ,sense= maximize)

            for i in range(No_Of_Products): # Constraints for the Products
                
                exec(f"Model.OverAll_Pn = Constraint(expr = sum(Total_Trade_Spent[0]) + sum(Total_Trade_Spent[1])  <= sum(Target_Trade_Spend))")
                exec(f"Model.OverAll_P{i*2+1} = Constraint(expr = sum(Total_Trade_Spent[{i}]) <= Target_Trade_Spend[{i}]*1.1)")
                exec(f"Model.OverAll_P{i*2+2} = Constraint(expr = sum(Total_Trade_Spent[{i}]) >= Target_Trade_Spend[{i}]*0.9)")
                exec(f"Model.EDLP_P{i*2+1} = Constraint(expr = sum(EDLP_Trade_Spent[{i}]) <= Target_EDLP_Spend[{i}]*1.2)" )
                exec(f"Model.EDLP_P{i*2+2} = Constraint(expr = sum(EDLP_Trade_Spent[{i}]) >= Target_EDLP_Spend[{i}]*0.8)" )
#                 exec(f"Model.TPR_P{i*2+1} = Constraint(expr = sum(TPR_Trade_Spent[{i}]) <= Target_TPR_Spend[{i}]*1.2)" )
#                 exec(f"Model.TPR_P{i*2+2} = Constraint(expr = sum(TPR_Trade_Spent[{i}]) >= Target_TPR_Spend[{i}]*0.8)" )
            
            
            start_time = time.time()
            opt_1 = SolverFactory(name)
            opt_1.solve(Model)
            end_time = time.time()
            print(f"\n SOLVER Name : {name} \n")
            print(f"\n P1 Initialization Value: {comb} , P2 Initialization value: {com} \n ")
            Model.display()
            print("\n\n The Elapsed Time is --- %s Seconds ---" % (end_time - start_time),"\n\n")



    'pyomo.core.base.constraint.SimpleConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.SimpleConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().

 SOLVER Name : bonmin 


 P1 Initialization Value: (0, 0, 0) , P2 Initialization value: (0, 0, 0) 
 
Model unknown

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :  9.673906401732486 :    10 : False : False : NonNegativeReals
          1 :   9.5 :  9.662604105010825 :    10 : False : False : NonNegativeReals
          2 :   9.5 :  9.993204317265258 :    10 : False : False : NonNegativeReals
          3 :   9.5 :   9.54332943390899 :    10 : False : False : NonNegativeReals
          4 :   9.5 :  9.993204317265258 :    10 : False : False : NonNegativeReals
          5 :   9.5 :  9.993204317265258 :    1


 SOLVER Name : bonmin 


 P1 Initialization Value: (0, 0, 0) , P2 Initialization value: (0, 1, 1) 
 
Model unknown

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :  9.692897825592702 :    10 : False : False : NonNegativeReals
          1 :   9.5 :  9.750541038420092 :    10 : False : False : NonNegativeReals
          2 :   9.5 :  9.643853733679734 :    10 : False : False : NonNegativeReals
          3 :   9.5 :  9.750541038420092 :    10 : False : False : NonNegativeReals
          4 :   9.5 :   9.76724412956868 :    10 : False : False : NonNegativeReals
          5 :   9.5 :  9.720482844592487 :    10 : False : False : NonNegativeReals
          6 :     5 : 38.639376581399596 :    50 : False : False : NonNegativeReals
          7 :     5 : 15.567858772764477 :    50 : False : False : NonNegativeReals
          8 :     5 : 16.158549013501197 :    50 : False : False : NonNegativeReals
    


 SOLVER Name : bonmin 


 P1 Initialization Value: (0, 0, 1) , P2 Initialization value: (0, 0, 0) 
 
Model unknown

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :  9.993204317265254 :    10 : False : False : NonNegativeReals
          1 :   9.5 :  9.662604105010825 :    10 : False : False : NonNegativeReals
          2 :   9.5 :  9.673906401732488 :    10 : False : False : NonNegativeReals
          3 :   9.5 :  9.993204317265254 :    10 : False : False : NonNegativeReals
          4 :   9.5 :  9.993204317265254 :    10 : False : False : NonNegativeReals
          5 :   9.5 :   9.54332943390899 :    10 : False : False : NonNegativeReals
          6 :     5 : 15.690714499386242 :    50 : False : False : NonNegativeReals
          7 :     5 : 27.500000224999997 :    50 : False : False : NonNegativeReals
          8 :     5 :  49.54807752384255 :    50 : False : False : NonNegativeReals
    


 SOLVER Name : bonmin 


 P1 Initialization Value: (0, 0, 1) , P2 Initialization value: (0, 1, 1) 
 
Model unknown

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :  9.692897825592702 :    10 : False : False : NonNegativeReals
          1 :   9.5 :  9.699033464264703 :    10 : False : False : NonNegativeReals
          2 :   9.5 :  9.643853733679736 :    10 : False : False : NonNegativeReals
          3 :   9.5 :  9.957377318672112 :    10 : False : False : NonNegativeReals
          4 :   9.5 :   9.76724412956868 :    10 : False : False : NonNegativeReals
          5 :   9.5 :  9.720482844592487 :    10 : False : False : NonNegativeReals
          6 :     5 :  25.98610100571491 :    50 : False : False : NonNegativeReals
          7 :     5 : 15.567858772764481 :    50 : False : False : NonNegativeReals
          8 :     5 :  49.49172192966773 :    50 : False : False : NonNegativeReals
    


 SOLVER Name : bonmin 


 P1 Initialization Value: (0, 1, 1) , P2 Initialization value: (0, 0, 0) 
 
Model unknown

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :  9.993204274736707 :    10 : False : False : NonNegativeReals
          1 :   9.5 :  9.662604105010827 :    10 : False : False : NonNegativeReals
          2 :   9.5 :  9.673906401732486 :    10 : False : False : NonNegativeReals
          3 :   9.5 :  9.588937256957033 :    10 : False : False : NonNegativeReals
          4 :   9.5 :  9.911062748042966 :    10 : False : False : NonNegativeReals
          5 :   9.5 :   9.54332943390899 :    10 : False : False : NonNegativeReals
          6 :     5 : 15.690714499386257 :    50 : False : False : NonNegativeReals
          7 :     5 :  49.54807746565515 :    50 : False : False : NonNegativeReals
          8 :     5 :  49.54807746565515 :    50 : False : False : NonNegativeReals
    

...


 SOLVER Name : bnb 


 P1 Initialization Value: (0, 1, 1) , P2 Initialization value: (0, 0, 0) 
 
Model unknown

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :               10.0 :    10 : False : False : NonNegativeReals
          1 :   9.5 :   9.67021521187527 :    10 : False : False : NonNegativeReals
          2 :   9.5 :   9.67021521187527 :    10 : False : False : NonNegativeReals
          3 :   9.5 :  9.534492005247571 :    10 : False : False : NonNegativeReals
          4 :   9.5 :               10.0 :    10 : False : False : NonNegativeReals
          5 :   9.5 :               10.0 :    10 : False : False : NonNegativeReals
          6 :     5 : 15.895697511931845 :    50 : False : False : NonNegativeReals
          7 :     5 :                5.0 :    50 : False : False : NonNegativeReals
          8 :     5 :                5.0 :    50 : False : False : NonNegativeReals
       


 SOLVER Name : bnb 


 P1 Initialization Value: (0, 1, 1) , P2 Initialization value: (0, 1, 1) 
 
Model unknown

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :               10.0 :    10 : False : False : NonNegativeReals
          1 :   9.5 :  9.662604107949118 :    10 : False : False : NonNegativeReals
          2 :   9.5 :   9.67390640445725 :    10 : False : False : NonNegativeReals
          3 :   9.5 :               10.0 :    10 : False : False : NonNegativeReals
          4 :   9.5 :                9.5 :    10 : False : False : NonNegativeReals
          5 :   9.5 :  9.543329437718342 :    10 : False : False : NonNegativeReals
          6 :     5 : 15.690714749583513 :    50 : False : False : NonNegativeReals
          7 :     5 :                5.0 :    50 : False : False : NonNegativeReals
          8 :     5 :                5.0 :    50 : False : False : NonNegativeReals
       


 SOLVER Name : bnb 


 P1 Initialization Value: (1, 1, 1) , P2 Initialization value: (0, 0, 0) 
 
Model unknown

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :   9.67021521187834 :    10 : False : False : NonNegativeReals
          1 :   9.5 :  9.707902080645454 :    10 : False : False : NonNegativeReals
          2 :   9.5 :    9.6702152118722 :    10 : False : False : NonNegativeReals
          3 :   9.5 :                9.5 :    10 : False : False : NonNegativeReals
          4 :   9.5 :  9.534492005247582 :    10 : False : False : NonNegativeReals
          5 :   9.5 :               10.0 :    10 : False : False : NonNegativeReals
          6 :     5 :                5.0 :    50 : False : False : NonNegativeReals
          7 :     5 : 15.895697511931823 :    50 : False : False : NonNegativeReals
          8 :     5 :                5.0 :    50 : False : False : NonNegativeReals
       

        Key  : Lower    : Body              : Upper
        None : 249174.0 : 249174.0000000001 :  None
    EDLP_P1 : Size=1
        Key  : Lower : Body              : Upper
        None :  None : 76039.20000000006 : 76039.2
    EDLP_P2 : Size=1
        Key  : Lower   : Body              : Upper
        None : 50692.8 : 76039.20000000006 :  None
    OverAll_Pn : Size=1
        Key  : Lower : Body               : Upper
        None :  None : 495104.30000000127 : 500433.0
    OverAll_P3 : Size=1
        Key  : Lower : Body               : Upper
        None :  None : 245930.30000000118 : 245930.30000000002
    OverAll_P4 : Size=1
        Key  : Lower    : Body               : Upper
        None : 201215.7 : 245930.30000000118 :  None
    EDLP_P3 : Size=1
        Key  : Lower : Body              : Upper
        None :  None : 42630.00000000125 : 42630.0
    EDLP_P4 : Size=1
        Key  : Lower   : Body              : Upper
        None : 28420.0 : 42630.00000000125 :  None


 The Elapsed


 SOLVER Name : bnb 


 P1 Initialization Value: (1, 1, 1) , P2 Initialization value: (1, 1, 1) 
 
Model unknown

  Variables:
    Variables : Size=18, Index=Slot
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :   9.5 :  9.643853736841315 :    10 : False : False : NonNegativeReals
          1 :   9.5 :  9.692897828005533 :    10 : False : False : NonNegativeReals
          2 :   9.5 :                9.5 :    10 : False : False : NonNegativeReals
          3 :   9.5 :  9.720482844325698 :    10 : False : False : NonNegativeReals
          4 :   9.5 :               10.0 :    10 : False : False : NonNegativeReals
          5 :   9.5 :  9.767244134437068 :    10 : False : False : NonNegativeReals
          6 :     5 :                5.0 :    50 : False : False : NonNegativeReals
          7 :     5 :                5.0 :    50 : False : False : NonNegativeReals
          8 :     5 : 15.567859019070925 :    50 : False : False : NonNegativeReals
       

In [20]:
# Guru Code for Buffer

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 10/04/20 3:35 PM
@author: Gurusankar G
"""
from __future__ import division
from pyomo.environ import *
import pyomo.environ as pyo
import itertools as it
from functools import reduce
import operator
import numpy as np

#TODO: Config data inputs.
# data inputs
base_price = [[10, 10, 10], [10, 10, 10]]

target_trade_spend = [276860, 223573]
target_edlp_spend = [63366, 35525]
target_tpr_spend = [213494, 188048]

intercept = [23, 20]
coef_1 = [-6, -5]  # edlp
coef_2 = [0.01, 0.02]  # tpr
coef_3 = [1, 1.2]  # compEDLP
coef_4 = [-0.003, -0.004]  # compTPR


def retailer_unit_sales(model, i, j):
    unit_sales = exp((log(model.edlp[i, j]) * coef_1[i]) * model.flag[i, j]
                     + (log(base_price[i][j]) * coef_1[i] + model.tpr[i, j] * coef_2[i]) * (1 - model.flag[i, j])
                     + intercept[i])

    unit_sales = unit_sales * competitor_unit_effect(model, i, j)
    return unit_sales


def retailer_price(model, i, j):
    price = (base_price[i][j] * (1 - model.tpr[i, j] / 100) * (1 - model.flag[i, j])) + (
            model.edlp[i, j] * model.flag[i, j])
    return price


def competitor_unit_effect(model, curr_ret, wk):
    comp_unit_sales_retailers = [exp((log(model.edlp[i, wk]) * coef_3[curr_ret]) * model.flag[i, wk] +
                                     (log(base_price[i][wk]) * coef_3[curr_ret] + model.tpr[i, wk] * coef_4[curr_ret]) *
                                     (1 - model.flag[i, wk])) for i in model.ppg_index if i != curr_ret]

    return reduce(operator.mul, comp_unit_sales_retailers, 1)


def calc_dollar_sales(model):
    return sum([retailer_unit_sales(model, ret, wk) * retailer_price(model, ret, wk) for wk in model.wk_index for ret in
                model.ppg_index])

#TODO: consolidate into get_value and constraints
def calc_ub_retailer_total_trade_spent(model, ret):
    return sum([(base_price[ret][wk] - retailer_price(model, ret, wk)) * retailer_unit_sales(model, ret, wk) for wk in
                model.wk_index]) <= target_trade_spend[ret]*(1+15/100)

def calc_lb_retailer_total_trade_spent(model, ret):
    return sum([(base_price[ret][wk] - retailer_price(model, ret, wk)) * retailer_unit_sales(model, ret, wk) for wk in
                model.wk_index]) >= target_trade_spend[ret]*(1-15/100)


def calc_overall_trade_spent(model):
    return sum([sum([(base_price[ret][wk] - retailer_price(model, ret, wk)) * retailer_unit_sales(model, ret, wk) for wk in
                model.wk_index]) for ret in model.ppg_index]) == sum(target_trade_spend)


def calc_ub_retailer_edlp_trade_spent(model, ret):
    return sum([(base_price[ret][wk] - retailer_price(model, ret, wk)) * retailer_unit_sales(model, ret, wk) *
                model.flag[ret, wk] for wk in model.wk_index]) <= target_edlp_spend[ret]*(1+20/100)

def calc_lb_retailer_edlp_trade_spent(model, ret):
    return sum([(base_price[ret][wk] - retailer_price(model, ret, wk)) * retailer_unit_sales(model, ret, wk) *
                model.flag[ret, wk] for wk in model.wk_index]) >= target_edlp_spend[ret]*(1-20/100)

def calc_ub_retailer_tpr_trade_spent(model, ret):
    return sum([(base_price[ret][wk] - retailer_price(model, ret, wk)) * retailer_unit_sales(model, ret, wk) *
                (1 - model.flag[ret, wk]) for wk in model.wk_index]) <= target_tpr_spend[ret]*(1+20/100)

def calc_lb_retailer_tpr_trade_spent(model, ret):
    return sum([(base_price[ret][wk] - retailer_price(model, ret, wk)) * retailer_unit_sales(model, ret, wk) *
                (1 - model.flag[ret, wk]) for wk in model.wk_index]) >= target_tpr_spend[ret]*(1-20/100)

#TODOend

def calc_flag_util(model, ret):
    flag_util = [model.flag[ret, i] * (1 - model.flag[ret, i]) for i in model.wk_index]
    return sum(flag_util) == 0


# TODO: make wk,ppgs part of initialization
def create_model(init):
    print("Init value= ", init)

    def initial(model, *index):
        return init[index]

    model = pyo.ConcreteModel(name='Spend_Optim')
    model.weeks = Param(initialize=3, domain=PositiveIntegers)
    model.ppgs = Param(initialize=2, domain=PositiveIntegers)
    model.wk_index = RangeSet(0, model.weeks - 1)
    model.ppg_index = RangeSet(0, model.ppgs - 1)
    # wk_index = range(wks)
    # ppg_index = range(ppgs)
    model.edlp = Var(model.ppg_index, model.wk_index, initialize=9.5, bounds=(9.5, 10), domain=NonNegativeReals)
    model.tpr = Var(model.ppg_index, model.wk_index, initialize=5, bounds=(5, 50), domain=NonNegativeReals)
    model.flag = Var(model.ppg_index, model.wk_index, initialize=initial, domain=Binary)

    #model.edlp_buffer = Var(model.ppg_index,initialize=0,bounds=(0,20),domain=NonNegativeReals)
    #model.tpr_buffer = Var(model.ppg_index,initialize=0,bounds=(0,20),domain=NonNegativeReals)
    #model.total_spend_buffer = Var(model.ppg_index,initialize=0,bounds=(0,15),domain=NonNegativeReals)

    model.obj = Objective(rule=calc_dollar_sales, sense=maximize)
    model.c1ub = Constraint(model.ppg_index, rule=calc_ub_retailer_total_trade_spent)
    model.c1lb = Constraint(model.ppg_index, rule=calc_lb_retailer_total_trade_spent)
    model.c2ub = Constraint(model.ppg_index, rule=calc_ub_retailer_edlp_trade_spent)
    model.c2lb = Constraint(model.ppg_index, rule=calc_lb_retailer_edlp_trade_spent)
    model.c3ub = Constraint(model.ppg_index, rule=calc_ub_retailer_tpr_trade_spent)
    model.c3lb = Constraint(model.ppg_index, rule=calc_lb_retailer_tpr_trade_spent)
    model.c4 = Constraint(model.ppg_index, rule=calc_flag_util)
    model.c5 = Constraint(rule=calc_overall_trade_spent)
    return model

def call_solver(init, ppgs, wks, name='bonmin'):
    model = create_model(init)
    opt_1 = SolverFactory(name)
    results_1 = opt_1.solve(model)
    print(pyomo.environ.value(model.obj))
    print([pyo.value(model.edlp[i, j]) for i in range(ppgs) for j in range(wks)])
    print([pyo.value(model.tpr[i, j]) for i in range(ppgs) for j in range(wks)])
    print([pyo.value(model.flag[i, j]) for i in range(ppgs) for j in range(wks)])
    print([(i, j) for i in range(ppgs) for j in range(wks)])
    model.display()
    

if __name__ == '__main__':
    wks = 3
    ppgs = 2
    combinations = it.product("01", repeat=wks * ppgs)

    for s in combinations:
        init = np.fromiter(s, np.int8).reshape(ppgs, wks)
        call_solver(init, ppgs=ppgs, wks=wks, name='bonmin')


Init value=  [[0 0 0]
 [0 0 0]]
5609291.313995466
[9.752450933052026, 9.5, 9.747549071947972, 9.752450933052026, 9.747549071947972, 9.544682000722691]
[8.538353452796967, 5.871164358838027, 9.027170766449473, 14.108779663010129, 8.758798081881553, 48.74346707690491]
[0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
Model Spend_Optim

  Variables:
    edlp : Size=6, Index=edlp_index
        Key    : Lower : Value             : Upper : Fixed : Stale : Domain
        (0, 0) :   9.5 : 9.752450933052026 :    10 : False : False : NonNegativeReals
        (0, 1) :   9.5 :               9.5 :    10 : False : False : NonNegativeReals
        (0, 2) :   9.5 : 9.747549071947972 :    10 : False : False : NonNegativeReals
        (1, 0) :   9.5 : 9.752450933052026 :    10 : False : False : NonNegativeReals
        (1, 1) :   9.5 : 9.747549071947972 :    10 : False : False : NonNegativeReals
        (1, 2) :   9.5 : 9.544682000722691 :    10 : False : False : NonNegativ

5601820.345965463
[9.77149505319214, 9.5, 9.728504951807858, 9.77149505319214, 10.0, 9.539925191456282]
[5.0, 5.6554495289781945, 12.248506867969999, 19.33163165856189, 48.69342534475858, 5.6554495289781945]
[0.0, 1.0, 0.0, 0.0, 1.0, 1.0]
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
Model Spend_Optim

  Variables:
    edlp : Size=6, Index=edlp_index
        Key    : Lower : Value             : Upper : Fixed : Stale : Domain
        (0, 0) :   9.5 :  9.77149505319214 :    10 : False : False : NonNegativeReals
        (0, 1) :   9.5 :               9.5 :    10 : False : False : NonNegativeReals
        (0, 2) :   9.5 : 9.728504951807858 :    10 : False : False : NonNegativeReals
        (1, 0) :   9.5 :  9.77149505319214 :    10 : False : False : NonNegativeReals
        (1, 1) :   9.5 :              10.0 :    10 : False : False : NonNegativeReals
        (1, 2) :   9.5 : 9.539925191456282 :    10 : False : False : NonNegativeReals
    tpr : Size=6, Index=tpr_index
        Key    : L

5601820.345965464
[9.728504951806032, 9.5, 9.771495053193966, 9.539925191456282, 10.0, 9.771495053193966]
[12.248506867969999, 5.655449528906675, 5.0, 5.655449528906675, 48.693425344898536, 19.331631658561893]
[0.0, 1.0, 0.0, 1.0, 1.0, 0.0]
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
Model Spend_Optim

  Variables:
    edlp : Size=6, Index=edlp_index
        Key    : Lower : Value             : Upper : Fixed : Stale : Domain
        (0, 0) :   9.5 : 9.728504951806032 :    10 : False : False : NonNegativeReals
        (0, 1) :   9.5 :               9.5 :    10 : False : False : NonNegativeReals
        (0, 2) :   9.5 : 9.771495053193966 :    10 : False : False : NonNegativeReals
        (1, 0) :   9.5 : 9.539925191456282 :    10 : False : False : NonNegativeReals
        (1, 1) :   9.5 :              10.0 :    10 : False : False : NonNegativeReals
        (1, 2) :   9.5 : 9.771495053193966 :    10 : False : False : NonNegativeReals
    tpr : Size=6, Index=tpr_index
        Key    :

5604487.698725589
[9.751102195802252, 9.748897809197747, 9.5, 9.748897809197747, 9.751102195802252, 9.535251376902048]
[8.763199492739702, 8.763199492739579, 5.942287895895749, 11.480696139000896, 11.480696139000921, 48.67361077436403]
[0.0, 0.0, 1.0, 0.0, 0.0, 1.0]
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
Model Spend_Optim

  Variables:
    edlp : Size=6, Index=edlp_index
        Key    : Lower : Value             : Upper : Fixed : Stale : Domain
        (0, 0) :   9.5 : 9.751102195802252 :    10 : False : False : NonNegativeReals
        (0, 1) :   9.5 : 9.748897809197747 :    10 : False : False : NonNegativeReals
        (0, 2) :   9.5 :               9.5 :    10 : False : False : NonNegativeReals
        (1, 0) :   9.5 : 9.748897809197747 :    10 : False : False : NonNegativeReals
        (1, 1) :   9.5 : 9.751102195802252 :    10 : False : False : NonNegativeReals
        (1, 2) :   9.5 : 9.535251376902048 :    10 : False : False : NonNegativeReals
    tpr : Size=6, Index=

5609291.313995465
[9.752450929561883, 9.747549075438116, 9.5, 9.544682000722691, 9.752450929561883, 9.747549075438116]
[9.027170766449569, 8.538353452796851, 5.871164556110245, 48.74346688314777, 14.108779663010129, 8.758798081881547]
[0.0, 0.0, 1.0, 1.0, 0.0, 0.0]
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
Model Spend_Optim

  Variables:
    edlp : Size=6, Index=edlp_index
        Key    : Lower : Value             : Upper : Fixed : Stale : Domain
        (0, 0) :   9.5 : 9.752450929561883 :    10 : False : False : NonNegativeReals
        (0, 1) :   9.5 : 9.747549075438116 :    10 : False : False : NonNegativeReals
        (0, 2) :   9.5 :               9.5 :    10 : False : False : NonNegativeReals
        (1, 0) :   9.5 : 9.544682000722691 :    10 : False : False : NonNegativeReals
        (1, 1) :   9.5 : 9.752450929561883 :    10 : False : False : NonNegativeReals
        (1, 2) :   9.5 : 9.747549075438116 :    10 : False : False : NonNegativeReals
    tpr : Size=6, Index=t

...

5648413.358860648
[9.694332045860788, 9.64222924168121, 9.61531694313426, 9.61531694313426, 9.722153949698091, 9.765563216445923]
[48.51909367389211, 48.51909367389211, 15.387811608355992, 19.342091796573346, 48.51909367389211, 48.51909367389211]
[1.0, 1.0, 0.0, 0.0, 1.0, 1.0]
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
Model Spend_Optim

  Variables:
    edlp : Size=6, Index=edlp_index
        Key    : Lower : Value             : Upper : Fixed : Stale : Domain
        (0, 0) :   9.5 : 9.694332045860788 :    10 : False : False : NonNegativeReals
        (0, 1) :   9.5 :  9.64222924168121 :    10 : False : False : NonNegativeReals
        (0, 2) :   9.5 :  9.61531694313426 :    10 : False : False : NonNegativeReals
        (1, 0) :   9.5 :  9.61531694313426 :    10 : False : False : NonNegativeReals
        (1, 1) :   9.5 : 9.722153949698091 :    10 : False : False : NonNegativeReals
        (1, 2) :   9.5 : 9.765563216445923 :    10 : False : False : NonNegativeReals
    tpr : Siz

5646529.990742032
[9.672238692992181, 9.672238692992181, 9.503655655072494, 9.746278441972624, 9.746278441972624, 9.503655655072494]
[46.38026120719794, 46.38026120719794, 15.866315860834721, 46.38026120719794, 46.38026120719794, 19.710172153283004]
[1.0, 1.0, 0.0, 1.0, 1.0, 0.0]
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
Model Spend_Optim

  Variables:
    edlp : Size=6, Index=edlp_index
        Key    : Lower : Value             : Upper : Fixed : Stale : Domain
        (0, 0) :   9.5 : 9.672238692992181 :    10 : False : False : NonNegativeReals
        (0, 1) :   9.5 : 9.672238692992181 :    10 : False : False : NonNegativeReals
        (0, 2) :   9.5 : 9.503655655072494 :    10 : False : False : NonNegativeReals
        (1, 0) :   9.5 : 9.746278441972624 :    10 : False : False : NonNegativeReals
        (1, 1) :   9.5 : 9.746278441972624 :    10 : False : False : NonNegativeReals
        (1, 2) :   9.5 : 9.503655655072494 :    10 : False : False : NonNegativeReals
    tpr : 

5661405.80990086
[9.669936494445508, 9.669936494445508, 9.750301133204774, 9.749698871795225, 9.750301133204774, 9.53477040173251]
[28.016731683727023, 28.016731683727023, 15.709640026524397, 11.375777605110407, 11.375777605110393, 28.016731683727023]
[1.0, 1.0, 0.0, 0.0, 0.0, 1.0]
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
Model Spend_Optim

  Variables:
    edlp : Size=6, Index=edlp_index
        Key    : Lower : Value             : Upper : Fixed : Stale : Domain
        (0, 0) :   9.5 : 9.669936494445508 :    10 : False : False : NonNegativeReals
        (0, 1) :   9.5 : 9.669936494445508 :    10 : False : False : NonNegativeReals
        (0, 2) :   9.5 : 9.750301133204774 :    10 : False : False : NonNegativeReals
        (1, 0) :   9.5 : 9.749698871795225 :    10 : False : False : NonNegativeReals
        (1, 1) :   9.5 : 9.750301133204774 :    10 : False : False : NonNegativeReals
        (1, 2) :   9.5 :  9.53477040173251 :    10 : False : False : NonNegativeReals
    tpr 

5661405.809900861
[9.749698950877633, 9.669936494445507, 9.669936494445508, 9.53477040173251, 9.749698950877633, 9.749698950877633]
[15.709640026524395, 28.01660679479466, 27.05758274236674, 27.05758274236674, 11.375777605110384, 11.375777605110441]
[0.0, 1.0, 1.0, 1.0, 0.0, 0.0]
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
Model Spend_Optim

  Variables:
    edlp : Size=6, Index=edlp_index
        Key    : Lower : Value             : Upper : Fixed : Stale : Domain
        (0, 0) :   9.5 : 9.749698950877633 :    10 : False : False : NonNegativeReals
        (0, 1) :   9.5 : 9.669936494445507 :    10 : False : False : NonNegativeReals
        (0, 2) :   9.5 : 9.669936494445508 :    10 : False : False : NonNegativeReals
        (1, 0) :   9.5 :  9.53477040173251 :    10 : False : False : NonNegativeReals
        (1, 1) :   9.5 : 9.749698950877633 :    10 : False : False : NonNegativeReals
        (1, 2) :   9.5 : 9.749698950877633 :    10 : False : False : NonNegativeReals
    tpr : 

5661749.030060757
[9.663195480749794, 9.749486843091601, 9.673018121125406, 9.749486843091601, 9.750513161908398, 9.543288826535981]
[33.12308370303846, 15.521167168522004, 33.12308370303846, 11.725273267122237, 11.228460969360972, 33.12308370303846]
[1.0, 0.0, 1.0, 0.0, 0.0, 1.0]
[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
Model Spend_Optim

  Variables:
    edlp : Size=6, Index=edlp_index
        Key    : Lower : Value             : Upper : Fixed : Stale : Domain
        (0, 0) :   9.5 : 9.663195480749794 :    10 : False : False : NonNegativeReals
        (0, 1) :   9.5 : 9.749486843091601 :    10 : False : False : NonNegativeReals
        (0, 2) :   9.5 : 9.673018121125406 :    10 : False : False : NonNegativeReals
        (1, 0) :   9.5 : 9.749486843091601 :    10 : False : False : NonNegativeReals
        (1, 1) :   9.5 : 9.750513161908398 :    10 : False : False : NonNegativeReals
        (1, 2) :   9.5 : 9.543288826535981 :    10 : False : False : NonNegativeReals
    tpr :